In [410]:
import numpy as np
import pandas as pd

In [411]:
!pip3 install pyvi

In [412]:
from pyvi import ViTokenizer, ViPosTagger

In [413]:
ViPosTagger.postagging(ViTokenizer.tokenize(u"Trường đại học Bách Khoa Hà Nội"))

(['Trường', 'đại_học', 'Bách_Khoa', 'Hà_Nội'], ['N', 'N', 'Np', 'Np'])

In [414]:
data = pd.read_csv('./reviews.csv')
data_sample = data.sample(frac = 1)

In [415]:
reviews = data_sample['Review']
labels = data_sample['Label'].fillna(0.)
reviews.head(10)
reviews.shape[0]

5914

In [416]:
englishwords = pd.read_json("./englishwords.json")
englishwords.set_index("english", inplace = True)
shortform = pd.read_json("./shortform.json")
shortform.set_index("short", inplace = True)
englishwords

,vietnamese
english,
fun,vui
phone,điện thoại
funny,hay ho
concept,ý tưởng
idea,ý tưởng
smartphone,điện thoại thông minh
no,không
yes,có
capable,có thể


In [417]:
shortform

,long
short,
hnay,hôm nay
k,không
ko,không
hqua,hôm qua
htrc,hôm trước
bh,bây giờ
bg,bây giờ
h,giờ
cn,công nghệ


In [418]:
reviews.loc[1300]

' Bạn bè bảo đi thử quán gia truyền nên đi Thề luôn chưa thấy quán bít tết nào tệ như điểm cũng dễ nhưng cầu thang di chuyển thì bé kiểu cổ ngày xưa bậc cao mà người đi phải lách Mình ngồi tầng nhưng nhìn chung chật nóng bàn kê san sát buổi tối mà đèn thì bật như kiểu tiết cảm tưởng như thời bao viên phục vụ Bàn người mà mang có đĩa dưa chuột bé gọi để hỏi thêm dưa chuột thì gọi mãi không thấy suất ăn của nó mới là quá kinh Mình gọi bánh mỳ chảo dăm bông trứng lòng đào thì mang cái chảo nhìn như đồ ăn trứng đến lòng trắng còn chảy nhão ra chưa thịt thì khô và Suất bít tết thì ngấy kinh một lượng mỡ ngập ngụa mà không có trứng cháy ở khoai tây lèo pate hộp nên cứng mà thịt bò thì dai thôi rồi cắt còn chẳng cắt hiểu vì sao quán vẫn khẩu vị tệ Thề không bao giờ quay trở'

In [419]:
# Loại bỏ kí tự đặc biệt và chuyển thành chữ cái viết thường
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    
    #Loại bỏ các chữ cái lặp nhau
    length = len(text)
    char = 0
    while char <length-1:
        if text[char] == text[char+1]:
            text = text[:char]+text[char+1:]
            #print(text)
            length-=1
            continue
        char+=1
    #loại bỏ số  và các chữ cái lạ:
    digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ(?::|;|=)(?:-)?(?:\)|\(|D|P)_ '
    text = ''.join([i for i in text if i in digits_and_characters])

    text_split = text.split()
    #print(text_split)
    for i in range(len(text_split)):
      t = text_split[i]
      if t in englishwords.index:
        text_split[i] = str(englishwords.loc[t, "vietnamese"])
      elif t in shortform.index:
        text_split[i] = str(shortform.loc[t,"long"]) 
    #print(text_split)
    text = ' '.join(text_split)
    return text


In [420]:
from pyvi import ViTokenizer

In [421]:
print(preprocessor('ngoonnnnnnnn quáaaaaaa trời ơiiii oiiii not yes ko <abcxyz> <345345345> |||| 12345맛있고'))

ngon quáa trời ơi oi not có không


In [422]:
def clean(series):
  for i in series.index:
    series.loc[i] = ViTokenizer.tokenize(preprocessor(series.loc[i]))

In [423]:
from collections import Counter

vocab = Counter()
for twit in reviews:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(10)

[('ăn', 9655),
 ('là', 7345),
 ('mình', 7256),
 ('có', 7248),
 ('thì', 7085),
 ('', 5913),
 ('không', 5839),
 ('và', 5715),
 ('cũng', 4675),
 ('mà', 3862)]

In [424]:
with open('./stopwords_vn_dash.txt') as f:
    stopwords_vn = f.read().splitlines()
stopwords_vn[:10]

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy']

In [425]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [426]:
from nltk.corpus import stopwords
stopwords_eng = stopwords.words('english')

In [427]:
stopwords = stopwords_vn + stopwords_eng

In [428]:
vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stopwords:
        vocab_reduced[w]=c

vocab_reduced.most_common(10)

[('', 5913),
 ('không', 5839),
 ('quán', 3742),
 ('viên', 2933),
 ('ngon', 2806),
 ('đi', 2772),
 ('ko', 2546),
 ('đồ', 2457),
 ('nhân', 2423),
 ('gọi', 2362)]

In [429]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( reviews, labels, test_size=0.3, random_state=42)

In [430]:
clean(X_train)

In [431]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(stop_words = stopwords_vn, min_df = 2, ngram_range= (1,2))
X_train_bow = bow.fit_transform(X_train)
print(X_train_bow.shape)
print(bow.get_feature_names())

(4139, 19375)
['_mình', 'abc', 'about', 'about the', 'ac', 'ace', 'ad', 'ae', 'ae chỗ', 'aen', 'aeon', 'aeon không', 'aeon mal', 'after', 'again', 'ahihi', 'ai_lại', 'ai_ngờ', 'ak', 'al', 'al fresco', 'al italian', 'alacarte', 'alfresco', 'almaz', 'alo', 'alo bàn', 'also', 'always', 'am', 'am in', 'amazing', 'an', 'an không', 'an ngon', 'an thi', 'an_toàn', 'an_toàn thực_phẩm', 'an_tâm', 'and', 'and also', 'and personaly', 'and realy', 'and the', 'and they', 'and this', 'and very', 'and văn_bản', 'and we', 'angel', 'angel in', 'anh_chị', 'anh_chị chủ', 'anh_chị không', 'anh_chị nhân_viên', 'anh_chị nv', 'anh_chị phục_vụ', 'anh_chị thông_cảm', 'anh_chị đi', 'anh_em', 'anh_em không', 'anh_em quán', 'anh_đào', 'any', 'ao', 'ap', 'are', 'are realy', 'aroi', 'around', 'as', 'asian', 'asked', 'asked for', 'at', 'at al', 'at least', 'ate', 'australia', 'australia bef', 'authentic', 'avalon', 'away', 'ba_chỉ', 'ba_chỉ bò', 'ba_chỉ heo', 'ba_chỉ không', 'ba_chỉ miếng', 'ba_chỉ ngon', 'ba_chỉ nư

In [432]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = stopwords_vn, min_df = 2, ngram_range= (1,2))
# Feed the tf-idf Vectorizer with twits using fit_transform()
X_train_tfidf = tfidf.fit_transform(X_train)

# Formatting the number to 2 digits after the decimal point by showing on this notebook
np.set_printoptions(precision=2)
# To print array in one line
np.set_printoptions(linewidth=np.inf)
print(tfidf.get_feature_names())
print(X_train_tfidf.shape)

['_mình', 'abc', 'about', 'about the', 'ac', 'ace', 'ad', 'ae', 'ae chỗ', 'aen', 'aeon', 'aeon không', 'aeon mal', 'after', 'again', 'ahihi', 'ai_lại', 'ai_ngờ', 'ak', 'al', 'al fresco', 'al italian', 'alacarte', 'alfresco', 'almaz', 'alo', 'alo bàn', 'also', 'always', 'am', 'am in', 'amazing', 'an', 'an không', 'an ngon', 'an thi', 'an_toàn', 'an_toàn thực_phẩm', 'an_tâm', 'and', 'and also', 'and personaly', 'and realy', 'and the', 'and they', 'and this', 'and very', 'and văn_bản', 'and we', 'angel', 'angel in', 'anh_chị', 'anh_chị chủ', 'anh_chị không', 'anh_chị nhân_viên', 'anh_chị nv', 'anh_chị phục_vụ', 'anh_chị thông_cảm', 'anh_chị đi', 'anh_em', 'anh_em không', 'anh_em quán', 'anh_đào', 'any', 'ao', 'ap', 'are', 'are realy', 'aroi', 'around', 'as', 'asian', 'asked', 'asked for', 'at', 'at al', 'at least', 'ate', 'australia', 'australia bef', 'authentic', 'avalon', 'away', 'ba_chỉ', 'ba_chỉ bò', 'ba_chỉ heo', 'ba_chỉ không', 'ba_chỉ miếng', 'ba_chỉ ngon', 'ba_chỉ nướng', 'ba_chỉ 

In [433]:
clean(X_test)

In [434]:
X_test_bow = bow.transform(X_test)
X_test_tfidf = tfidf.transform(X_test)

In [435]:
from sklearn.naive_bayes import MultinomialNB
nbc = MultinomialNB()

In [436]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [437]:
nbc.fit(X_train_bow.toarray(), y_train)
y_pred = nbc.predict(X_test_bow.toarray())

In [438]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7656338028169014
0.5488071796932555


In [439]:
nbc.fit(X_train_tfidf.toarray(), y_train)
y_pred = nbc.predict(X_test_tfidf.toarray())

In [440]:
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7740845070422535
0.5504358339962682


In [441]:
from sklearn.linear_model import LogisticRegression


In [442]:
lor = LogisticRegression()
lor.fit(X_train_bow.toarray(), y_train)
y_pred = lor.predict(X_test_bow.toarray())
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7667605633802816
0.6091741239168775


In [443]:
lor.fit(X_train_tfidf.toarray(), y_train)
y_pred = lor.predict(X_test_tfidf.toarray())
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))
print(classification_report(y_test, y_pred))

0.783661971830986
0.5685707804719945
              precision    recall  f1-score   support

        -1.0       0.78      0.86      0.82       734
         0.0       0.36      0.02      0.03       226
         1.0       0.79      0.93      0.85       815

    accuracy                           0.78      1775
   macro avg       0.65      0.60      0.57      1775
weighted avg       0.73      0.78      0.73      1775



In [444]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth = None, random_state = 0)

In [445]:
rfc.fit(X_train_bow.toarray(), y_train)
y_pred = rfc.predict(X_test_bow.toarray())
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7628169014084507
0.5535635599092928


In [446]:
rfc.fit(X_train_tfidf.toarray(), y_train)
y_pred = rfc.predict(X_test_tfidf.toarray())
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average = 'macro'))

0.7695774647887323
0.5560418287935326


In [447]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

In [448]:
mlp.fit(X_train_bow, y_train)
y_pred = mlp.predict(X_test_bow)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.78      0.77      0.77       734
         0.0       0.28      0.18      0.22       226
         1.0       0.79      0.87      0.82       815

    accuracy                           0.74      1775
   macro avg       0.61      0.61      0.61      1775
weighted avg       0.72      0.74      0.73      1775



In [449]:
mlp.fit(X_train_tfidf, y_train)
y_pred = mlp.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.78      0.81      0.79       734
         0.0       0.27      0.11      0.16       226
         1.0       0.78      0.89      0.83       815

    accuracy                           0.76      1775
   macro avg       0.61      0.60      0.59      1775
weighted avg       0.72      0.76      0.73      1775



In [450]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler(with_mean= False)
X_train_bow = ss.fit_transform(X_train_bow)
X_test_bow = ss.transform(X_test_bow)
X_train_tfidf = ss.fit_transform(X_train_tfidf)
X_test_tfidf  = ss.transform(X_test_tfidf)

In [451]:
from sklearn.svm import SVC
clf_svc = SVC(C = 1.)



In [452]:
clf_svc.fit(X_train_bow, y_train)
y_pred = clf_svc.predict(X_test_bow)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.77      0.50      0.61       734
         0.0       0.00      0.00      0.00       226
         1.0       0.59      0.94      0.72       815

    accuracy                           0.64      1775
   macro avg       0.45      0.48      0.44      1775
weighted avg       0.59      0.64      0.58      1775



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [453]:
clf_svc.fit(X_train_tfidf, y_train)
y_pred = clf_svc.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.74      0.79      0.76       734
         0.0       0.00      0.00      0.00       226
         1.0       0.73      0.88      0.80       815

    accuracy                           0.73      1775
   macro avg       0.49      0.56      0.52      1775
weighted avg       0.64      0.73      0.68      1775



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
